<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/Eyelid_COCO_annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Eyelid_COCO_annotation**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from PIL import Image # (pip install Pillow)
import numpy as np                                 # (pip install numpy)
from skimage import measure                        # (pip install scikit-image)
from shapely.geometry import Polygon, MultiPolygon # (pip install Shapely)

def create_sub_masks(mask_image):
    width, height = mask_image.size

    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))[:3]

            # If the pixel is not black...
            if pixel != (0, 0, 0):
                # Check to see if we've created a sub-mask...
                pixel_str = str(pixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks


def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')

    segmentations = []
    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    x, y, max_x, max_y = multi_poly.bounds
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation

In [ ]:
import json

plant_book_mask_image = Image.open('/content/Images/1/1_1i_Ll_1.jpg')
bottle_book_mask_image = Image.open('/content/Images/1/1_1i_Ll_2.jpg')

mask_images = [plant_book_mask_image, bottle_book_mask_image]

# Define which colors match which categories in the images
houseplant_id, book_id, bottle_id, lamp_id = [1, 2, 3, 4]
category_ids = {
    1: {
        '(0, 255, 0)': houseplant_id,
        '(0, 0, 255)': book_id,
    },
    2: {
        '(255, 255, 0)': bottle_id,
        '(255, 0, 128)': book_id,
        '(255, 100, 0)': lamp_id,
    }
}

is_crowd = 0

# These ids will be automatically increased as we go
annotation_id = 1
image_id = 1

# Create the annotations
annotations = []
for mask_image in mask_images:
    sub_masks = create_sub_masks(mask_image)
    for color, sub_mask in sub_masks.items():
        category_id = category_ids[image_id][color]
        annotation = create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd)
        annotations.append(annotation)
        annotation_id += 1
    image_id += 1

print(json.dumps(annotations))

In [ ]:
!unzip "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS.zip" -d "/content/"


In [5]:
import json
import collections as cl
from skimage import measure
import numpy as np
import cv2
import os

# Redefine the functions with corrections and complete them with the necessary steps

def info():
    tmp = cl.OrderedDict()
    tmp["description"] = "Test"
    tmp["url"] = "https://test"
    tmp["version"] = "0.01"
    tmp["year"] = 2020
    tmp["contributor"] = "salt22g"
    tmp["data_created"] = "2020/12/20"
    return tmp

def licenses():
    tmp = cl.OrderedDict()
    tmp["id"] = 1
    tmp["url"] = "dummy_words"
    tmp["name"] = "salt22g"
    return tmp

def images(file_path):
    tmps = []
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    height, width = img.shape[:2]

    tmp = cl.OrderedDict()
    tmp["license"] = 1
    tmp["id"] = 0
    tmp["file_name"] = os.path.basename(file_path)
    tmp["width"] = width
    tmp["height"] = height
    tmp["date_captured"] = ""
    tmp["coco_url"] = "dummy_words"
    tmp["flickr_url"] = "dummy_words"
    tmps.append(tmp)
    return tmps

def annotations(file_path):
    tmps = []
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    # Use the blue channel for finding contours which is the pupil (assuming red for sclera and green for iris)
    blue_channel = img[:,:,0]
    contours = measure.find_contours(blue_channel, 0.5)

    for i, contour in enumerate(contours):
        # Flip from (row, col) to (x, y) and then flatten
        contour = np.fliplr(contour).flatten().tolist()
        segmentation_list = contour

        # Create bounding box from the segmentation
        xs = contour[0::2]
        ys = contour[1::2]
        xmin = min(xs)
        xmax = max(xs)
        ymin = min(ys)
        ymax = max(ys)
        width = xmax - xmin
        height = ymax - ymin
        bbox = [xmin, ymin, width, height]
        area = width * height

        tmp = cl.OrderedDict()
        tmp["segmentation"] = [segmentation_list]
        tmp["id"] = str(i)
        tmp["image_id"] = 0
        tmp["category_id"] = 1
        tmp["area"] = area
        tmp["iscrowd"] = 0
        tmp["bbox"] = bbox
        tmps.append(tmp)
    return tmps

def categories():
    tmps = []
    sup = ["person"]
    cat = ["person"]
    for i in range(len(sup)):
        tmp = cl.OrderedDict()
        tmp["id"] = i+1
        tmp["name"] = cat[i]
        tmp["supercategory"] = sup[i]
        tmps.append(tmp)
    return tmps

def main(img_path, json_name):
    query_list = ["info", "licenses", "images", "annotations", "categories"]
    js = cl.OrderedDict()
    for query in query_list:
        if query == "info":
            js[query] = info()
        elif query == "licenses":
            js[query] = licenses()
        elif query == "images":
            js[query] = images(img_path)
        elif query == "annotations":
            js[query] = annotations(img_path)
        elif query == "categories":
            js[query] = categories()

    # write
    with open(json_name, 'w') as fw:
        json.dump(js, fw, indent=2)

# The path to the image and the name of the json file
img_path = '/content/Images/1/1_1i_Ll_1.jpg'
json_name = '/content/coco_format.json'

# Call main function with provided arguments
main(img_path, json_name)

# Return the path to the created json file
json_name


'/content/coco_format.json'